# LEAFToolbox for Sites
Applies the LEAF-ToolBox for mapping vegetation using satellite imagery over a list of sites, each defined using time stamped vector geographical objects.
Cite as Fernandes, R. et al., 2021, "LEAF Toolbox", Canada Centre for Remote Sensing, https://github.com/rfernand387/LEAF-Toolbox/wiki, DOI: 10.5281/zenodo.4321298.


LEAF currently supports two algorithms: 

SL2PV0 is the algorithm defined by Weiss and Baret (2016) with an update in Weiss and Baret (2020).  


SL2PV1.1 defined in  Fernandes et al. 2024 that attempts to correct for biases in crown and shoot clumping over forests observed in SL2PV0.


Refer to https://github.com/rfernand387/LEAF-Toolbox/tree/master/Source-Python for configuration of anaconda environment.


Weiss, M. and Baret, F. 2016. S2ToolBox Level 2 products: LAI, FAPAR, FCOVER, 1.1. ed.
Institut National de la Recherche Agronomique, Avignon, France. https://step.esa.
int/docs/extra/ATBD_S2ToolBox_L2B_V1.1.pdf.

Weiss, M., and Baret, F., 2020. S2ToolBox Level 2 Products: LAI, FAPAR, FCOVER, 2.0. ed.
Institut National de la Recherche Agronomique, Avignon, France. https://step.esa.
int/docs/extra/ATBD_S2ToolBox_L2B_V2.0.pdf.

Fernandes, R., Canisius, F., Harvey, K., Hong, G., MacDougall, C., Shah, H., Sun, L., 2024. Evidence of a bias-variance trade off when correcting for bias in Sentinel 2 forest LAI retrievals using radiative transfer models. Remote Sensing of EnvironmentRemote Sens. Environ. 305. doi:10.1016/j.rse.2024.114060.


## Setup Environment

In [3]:
# Provide your GEE authentificaton
import ee
ee.Authenticate(force=True)

Enter verification code:  4/1AVGzR1DHkapIFp4BKggFAxwdmkkuDubipJiAF9x_wIl9xPRexb3ZJvHxy0o



Successfully saved authorization token.


In [4]:
ee.Initialize()

In [33]:
# import LEAF modules
import LEAF

# # import algorithm definitions
import SL2PV0 
import SL2PV1
import SL2PV1MIX
import SL2PV1DBF
import SL2PV1ENF

## Run LEAF 

In [14]:
# Function definition
#
# status = LEAF.sampleSites(["projects/ee-modis250/assets/Anderson/Anderson_August2018"],
#                                     sensorName = 'S2', 
#                                     imageCollectionName="COPERNICUS/S2_SR_HARMONIZED",
#                                     outputPath='LEAFAnderson',
#                                     algorithm=SL2PV0, \
#                                     variableName="LAI",
#                                     featureRange=[0,np.nan], 
#                                     maxCloudcover=80,
#                                     outputScale=20,
#                                     exportDevice='CloudStorage',
#                                     prefixProperty='PlotID',
#                                     inputScale=20,
#                                     bufferSpatialSize=60,
#                                     subsamplingFraction=1,
#                                     numPixels=0,
#                                     bufferTemporalSize=[-10,10],
#                                     calendarRange=[1,12,'month'],
#                                     timeZone='Etc/GMT+6',
#                                     shardSize='YS')
#
# Applies a LEAF toolbox algorithm to map a canopy variable for all clear sky unmasked pixels from an input image collection
# falling within the spatial and temporal extents of features in a list of sites.  The spatial extent of features is defined by their GEE geometry.  
# The temporal extent of features is defined by their "system:time_start" and "system:time_end" properties or user defined otherwise.
#
# Parameters:
#
# siteList: list of readable GEE feature collection assets.  Features must have a system:time_start and system:time_end property i
# sensorName: User provided sensor name
# imageCollectionName: input image collection name from one of the list defined by GEE
#                 ["COPERNICUS/S2_SR_HARMONIZED","LANDSAT/LC08/C02/T1_L2","LANDSAT/LC09/C02/T1_L2","NASA/HLS/HLSL30/v002"]
# outputPathName: name of path (for Google Drive, GEE Asset) or bucket (for Google Cloud Platform Cloud Storage)
# algorithm: python module specifying LEAF algorithm to apply , currently one of list ["SL2PV0","SL2PV1","SL2PV1ENF","SL2PV1DBF","SL2PV1MIX"]
# variableName: variable to be mapped from one of the list ["ALBEDO","FAPAR","FCOVER","LAI","CWC","CCC","DASF"] defined by https://github.com/rfernand387/LEAF-Toolbox/wiki/Visualisation-Outputs        
# feature_range: range of features in each site GEE feature collection to sample, use [0,np.nan] for all    
# maxCloudCover: maximum input image cloud cover percentage [0,100]
# outputScale: scale (m) applied to products after generating them using inputScale and before exporting (refer to https://developers.google.com/earth-engine/guides/scale)
# exportDevice: either "Drive" for Google Drive, "Asset" for GEE Asset, or "CloudStorage" for Google Cloud Platform
# prefixProperty: property name of features in siteList to use in output name as a unique identifier
# inputScaleSize: scale (m) applied to input imagery before applying algorithm (refer to https://developers.google.com/earth-engine/guides/scale)
# bufferSpatialSize: width (m) of spatial buffer applied to features prior to sampling
# subsamplingFraction: fraction (0 to 1) of pixels to sample within a feature on any given image
# numPixels: maximum number of pixels to sample within a feature, overrides subsampling fraction
# bufferTemporalSize: Either dilation (days) of  date range specified by each system:time_start and system:time_end property OR temporal interval ["YYYY-MM-DD","YYYY-MM-DD"] that overrides any feature specific intervals
# calendarRange: [MM,MM,'month'] annual calendar range to restrict processing
# timeZone: GEE time zone for all date computations https://developers.google.com/earth-engine/apidocs/ee-date-format
# shardSize: size of temporal shards used for output, defaulted as individual years

                                                                                                           
# Output:
#
# One .CSV file for each feature for each shard (usually years spanning time period requested) with one row per sample per date 
# Columns:
# system:index - feature systemLindex property
# QC - LEAF QC flag , 0 indicates valid, else invalid
# date - UNIX epoch time stamp (milliseconds since 1970-01-01) for time zone specified
# errorXX - LEAF precision error for quantity XX
# estimateXX - LEAF estimate for quantity xx
# latitude - Latitude of product output pixel centroid , be sure to remember output scale
# longitude - Longitude of product outpixel centroid, be sure to remember output scale
# networkID - Neural network used for retrieval , this will vary with algorithm used
# partition - Partition used to specify neural networks, degfaults to NALCMS land cover legend
# s2cloudless_probability - S2Cloudless probability, only for "COPERNICUS/S2_SR_HARMONIZED"
# .geo - geographic object for output sample, currently defaulted to a point



In [ ]:
# Function definition
#
# status = LEAF.sampleImages(["projects/ee-modis250/assets/Anderson/Anderson_August2018"],
#                                     sensorName = 'S2', 
#                                     imageCollectionName="COPERNICUS/S2_SR_HARMONIZED",
#                                     outputPath='LEAFAnderson',
#                                     algorithm=SL2PV0, \
#                                     variableName="LAI",
#                                     featureRange=[0,np.nan], 
#                                     maxCloudcover=80,
#                                     outputScale=20,
#                                     exportDevice='CloudStorage',
#                                     prefixProperty='PlotID',
#                                     inputScale=20,
#                                     bufferSpatialSize=60,
#                                     subsamplingFraction=1,
#                                     numPixels=0,
#                                     bufferTemporalSize=[-10,10],
#                                     calendarRange=[1,12,'month'],
#                                     timeZone='Etc/GMT+6',
#                                     shardSize='YS')
#
# Applies a LEAF toolbox algorithm to map a canopy variable for all clear sky unmasked pixels from an input image collection
# falling within the spatial and temporal extents of features in a list of sites.  The spatial extent of features is defined by their GEE geometry.  
# The temporal extent of features is defined by their "system:time_start" and "system:time_end" properties or user defined otherwise.
#
# Parameters:
#
# siteList: list of readable GEE feature collection assets.  Features must have a system:time_start and system:time_end property i
# sensorName: User provided sensor name
# imageCollectionName: input image collection name from one of the list defined by GEE
#                 ["COPERNICUS/S2_SR_HARMONIZED","LANDSAT/LC08/C02/T1_L2","LANDSAT/LC09/C02/T1_L2","NASA/HLS/HLSL30/v002"]
# outputPathName: name of path (for Google Drive, GEE Asset) or bucket (for Google Cloud Platform Cloud Storage)
# algorithm: python module specifying LEAF algorithm to apply , currently one of list ["SL2PV0","SL2PV1","SL2PV1ENF","SL2PV1DBF","SL2PV1MIX"]
# variableName: variable to be mapped from one of the list ["ALBEDO","FAPAR","FCOVER","LAI","CWC","CCC","DASF"] defined by https://github.com/rfernand387/LEAF-Toolbox/wiki/Visualisation-Outputs        
# feature_range: range of features in each site GEE feature collection to sample, use [0,np.nan] for all    
# maxCloudCover: maximum input image cloud cover percentage [0,100]
# outputScale: scale (m) applied to products after generating them using inputScale and before exporting (refer to https://developers.google.com/earth-engine/guides/scale)
# exportDevice: either "Drive" for Google Drive, "Asset" for GEE Asset, or "CloudStorage" for Google Cloud Platform
# prefixProperty: property name of features in siteList to use in output name as a unique identifier
# inputScaleSize: scale (m) applied to input imagery before applying algorithm (refer to https://developers.google.com/earth-engine/guides/scale)
# bufferSpatialSize: width (m) of spatial buffer applied to features prior to sampling
# subsamplingFraction: fraction (0 to 1) of pixels to sample within a feature on any given image
# numPixels: maximum number of pixels to sample within a feature, overrides subsampling fraction
# bufferTemporalSize: Either dilation (days) of  date range specified by each system:time_start and system:time_end property OR temporal interval ["YYYY-MM-DD","YYYY-MM-DD"] that overrides any feature specific intervals
# calendarRange: [MM,MM,'month'] annual calendar range to restrict processing
# timeZone: GEE time zone for all date computations https://developers.google.com/earth-engine/apidocs/ee-date-format
# shardSize: size of temporal shards used for output, defaulted as individual years

                                                                                                           
# Output:
#
# One geotiff file for each feature for each shard (usually years spanning time period requested) with one row per sample per date 
# Layers:
# system:index - feature systemLindex property
# QC - LEAF QC flag , 0 indicates valid, else invalid
# date - UNIX epoch time stamp (milliseconds since 1970-01-01) for time zone specified
# errorXX - LEAF precision error for quantity XX
# estimateXX - LEAF estimate for quantity xx
# latitude - Latitude of product output pixel centroid , be sure to remember output scale
# longitude - Longitude of product outpixel centroid, be sure to remember output scale
# networkID - Neural network used for retrieval , this will vary with algorithm used
# partition - Partition used to specify neural networks, degfaults to NALCMS land cover legend
# s2cloudless_probability - S2Cloudless probability, only for "COPERNICUS/S2_SR_HARMONIZED"
# .geo - geographic object for output sample, currently defaulted to a point

### Run LEAF for 60m radius buffer around each site for +/-10d of field survey date using S2 data and output as CSV on GCP

In [34]:
status = LEAF.sampleSites(["projects/ee-modis250/assets/Anderson/Anderson_August2018"],
                                    sensorName = 'S2x', 
                                    imageCollectionName="COPERNICUS/S2_SR_HARMONIZED",
                                    outputPath='LEAFAnderson',
                                    algorithm=SL2PV0, \
                                    variableName="LAI",
                                    featureRange=[0,30], 
                                    maxCloudcover=80,
                                    outputScale=20,
                                    exportDevice='CloudStorage',
                                    prefixProperty='PlotID',
                                    inputScale=20,
                                    bufferSpatialSize=60,
                                    subsamplingFraction=1,
                                    numPixels=0,
                                    bufferTemporalSize=[-10,10],
                                    calendarRange=[1,12,'month'],
                                    timeZone='Etc/GMT+6',
                                    shardSize='YS')

STARTING LEAF.sampleSites using  COPERNICUS/S2_SR_HARMONIZED
Site:  projects/ee-modis250/assets/Anderson/Anderson_August2018  with  30  features.


NameError: name 'outputDevice' is not defined

### Run LEAF for 60m radius buffer around each site for April to September 2018 using S2 data and output as CSV on GCP

In [22]:
 status = LEAF.sampleSites(["projects/ee-modis250/assets/Anderson/Anderson_August2018"]
                                    sensorName = 'S2', 
                                    imageCollectionName="COPERNICUS/S2_SR_HARMONIZED",
                                    outputPath='LEAFAnderson',
                                    algorithm=SL2PV0, \
                                    variableName="LAI",
                                    featureRange=[0,30], 
                                    maxCloudcover=80,
                                    outputScale=20,
                                    exportDevice='CloudStorage',
                                    prefixProperty='PlotID',
                                    inputScale=20,
                                    bufferSpatialSize=60,
                                    subsamplingFraction=1,
                                    numPixels=0,
                                    bufferTemporalSize=["2018-03-01","2019-03-01"],
                                    calendarRange=[4,9,'month'],
                                    timeZone='Etc/GMT+6',
                                    shardSize='YS')

STARTING LEAF.sampleSites using  COPERNICUS/S2_SR_HARMONIZED
Site:  projects/ee-modis250/assets/Anderson/Anderson_May2018  with  30  features.
Processing Features: from 0 to 30
Feature n°: 0/30  -- startDate: 2018-03-01 00:00:00 -- endDate: 2019-03-01 00:00:00
----------------------------------------------------------------------------------------------------------
Sampling from  2018-03-01 00:00:00  to  2019-03-01 00:00:00
Export task started with ID: TDOAQ2FQTCSUY6EARXOLLG6M
Feature n°: 1/30  -- startDate: 2018-03-01 00:00:00 -- endDate: 2019-03-01 00:00:00
----------------------------------------------------------------------------------------------------------
Sampling from  2018-03-01 00:00:00  to  2019-03-01 00:00:00
Export task started with ID: KYNH6MAXGMLUYAEKELQNLHL7
Feature n°: 2/30  -- startDate: 2018-03-01 00:00:00 -- endDate: 2019-03-01 00:00:00
----------------------------------------------------------------------------------------------------------
Sampling from  2018-03

### Run LEAF for Study Region for July and August 2018 and output as geotiff on GCP

In [29]:
HLSL30LAIV0_2025= LEAF.imageSites(["projects/ee-modis250/assets/Anderson/AndersonRegion"], \
                                    imageCollectionName="COPERNICUS/S2_SR_HARMONIZED",\
                                    sensorName = 'S2', \
                                    outputPath='LEAFAnderson',
                                    algorithm=SL2PV0, \
                                    variableName="LAI",\
                                    featureRange=[0,1], \
                                    maxCloudcover=80,\
                                    outputScale=20,\
                                    exportDevice='CloudStorage',
                                    prefixProperty='system:index',
                                    inputScale=20,
                                    bufferSpatialSize=0,
                                    bufferTemporalSize=['2018-03-01','2019-03-01'],
                                    calendarRange=[7,8,'month'],
                                    timeZone='Etc/GMT+6',
                                    shardSize='YS')

STARTING LEAF.imageSites using  COPERNICUS/S2_SR_HARMONIZED
Site:  projects/ee-modis250/assets/Anderson/AndersonRegion  with  1  features.
Processing Features: from 0 to 1
Feature n°: 0/1  -- startDate: 2018-03-01 00:00:00 -- endDate: 2019-03-01 00:00:00
----------------------------------------------------------------------------------------------------------
Mapping from  2018-03-01 00:00:00  to  2019-03-01 00:00:00
Export task started with ID: E7BZWE7KO2GY3VLAPYN7HLVZ
Export task started with ID: CDOOMC476XMW4J4XCYRMWWQP
Export task started with ID: REMENDIA7ENBJRCZMWBQKYVT
Export task started with ID: ID73YX2UBA4ECYEGMZCUD3EP
Export task started with ID: HGNDRMDMCWZMPPUWY2VUS76T
Export task started with ID: 2EZOQGSEPOWZSTUMFGWJCKH7
Export task started with ID: CDMQTXGMZFJ7M5I4OEGU2A4T
Export task started with ID: GJ2ETSEIPGVCVZMSJGFQDFIY
Export task started with ID: XNYAEXD4EWJVRBG7HDLFKYX5
Export task started with ID: UU5SHCM6T4WRMPVWINKGPU7R
Export task started with ID: CAYW3KXMKE